<h1>Interaction with Claude API</h1>


This notebook introduced Claud API capabilities.<br>
This notebook started from codes in <a href="https://anthropic.skilljar.com/claude-with-the-anthropic-api/" target="_blank">Anthrophic Building with the Claude API</a> course.
<br><br>
Demonstrates
- How context influences generated completion
- How **system prompt** influences generated completion
- How **assistant prompt** influences generated completion
- How **temperature** (low,high) influences generated completion
- How generated completions consumed via **streaming** interface
- How **stop_sequences** stops generation in addition to **max_tokens**

<h2>Setup</h2>
Install anthropic and python-dotenv libraries<br>
python-dotenv allows program to load configuration data from .env file. <br>

In [ ]:
%pip install anthropic python-dotenv

AntrophicPrompt class keeps track of conversation history to support the context. <br>
Class accumulates user and assistant prompts.<br>
There is only one system prompt. <br>
Method clear_history() clears all accumulated prompts. <br>
Default model is "claude-3-5-haiku-latest"<br>

In [2]:
from anthropic import Anthropic

class AntrophicPrompt():

    def __init__(self, model="claude-3-5-haiku-latest"):
        self.model = model
        self.client = Anthropic()
        self.messages = []
        self.user_message = None
        self.assistant_message = None
        self.system_message = None

    def add_user_message(self, text):
        self.user_message = {"role": "user", "content": text}
        self.messages.append(self.user_message)


    def add_assistant_message(self, text):
        self.assistant_message = {"role": "assistant", "content": text}
        self.messages.append(self.assistant_message)


    def set_system_message(self, text):
        self.system_message = text

    def gen_completion(self, system=None, temperature=1.0, stop_sequences=[]):
        params = {
            "model": self.model,
            "max_tokens": 1000,
            "messages": self.messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }

        if system:
            params["system"] = system
        elif self.system_message:
            params["system"] = self.system_message

        message = self.client.messages.create(**params)
        return message.content[0].text

    def clear_history(self):
        self.messages = []
        self.user_message = None
        self.assistant_message = None
        self.system_message = None
        self.model = "claude-3-5-haiku-latest"

Get API key (from .env file)

In [3]:
# Load API Key
from dotenv import load_dotenv
load_dotenv()

True

Let's test by generating the first completion.

In [4]:
#
model = "claude-sonnet-4-0"
client = AntrophicPrompt(model)

user_prompt = "Define quantum computing in one sentence"
client.add_user_message(user_prompt)

#Get Completion
answer = client.gen_completion()
print(answer)

Quantum computing is a type of computation that harnesses quantum mechanical phenomena like superposition and entanglement to process information in ways that can potentially solve certain problems exponentially faster than classical computers.


The following example demonstrates how the **context** helps. <br>
The user prompt is "Write another sentence" and this prompt without a context is very ambiguous. <br>
Adding the prior answer as "assistant prompt" established a context to generate for the given "user prompt". <br>

In [17]:
client.add_assistant_message(answer)
user_prompt = "Write another sentence"
client.add_user_message(user_prompt)
answer = client.gen_completion()
print(answer)

Quantum computers use quantum bits (qubits) that can exist in multiple states simultaneously, unlike classical bits that are limited to just 0 or 1, enabling them to perform many calculations in parallel.


The following example demonstrates how to implement conversation via Claude API <br>
- What is 2+3?
- Add 7 to that
- Divide by 2 and if the result is even print "even" otherwise print "odd"

In [25]:
client.clear_history()
i=0
while i < 3:
    user_input=input("?>")
    print("user_input: "+user_input)
    client.add_user_message(user_input)
    answer = client.gen_completion()
    client.add_assistant_message(answer)
    print("------------")
    print(answer)
    print("------------")
    i += 1

user_input: What is 2+5?
------------
7
------------
user_input: Add 7 to that
------------
7 + 7 = 14
------------
user_input: Divide by 2 and if the result is even print "even" otherwise print "odd"
------------
Let me solve this step by step:

1. 14 ÷ 2 = 7
2. 7 is an odd number

So, the result is "odd"
------------


<h2>How does system prompt influence the generated completion?</h2>
The effect of "system prompt" when generating a completion.<br>
Here it is used to assign a role to LLM <br>

In [33]:
# Start a new conversation
client.clear_history()
user_prompt = "How to solve x^2+2x=-1 for x?"
client.add_user_message(user_prompt)

# Without system prompt
answer = client.gen_completion()
print(f"Answer without system prompt : "+answer)

# With system prompt
system = """
You are a patient math tutor.
Do not directly answer a student's questions.
Guide them to a solution step by step.
"""
client.set_system_message(system)
answer2 = client.gen_completion()
print(f"Answer with system prompt : "+answer2)

Answer without system prompt : Let's solve this step by step:

1) First, rearrange the equation to standard quadratic form by moving all terms to one side:
   x^2 + 2x + 1 = 0

2) This is a quadratic equation in the form ax^2 + bx + c = 0
   Where a = 1, b = 2, and c = 1

3) We can solve this using the quadratic formula: x = [-b ± √(b^2 - 4ac)] / (2a)

4) Substitute the values:
   x = [-2 ± √(2^2 - 4(1)(1))] / (2(1))

5) Simplify under the square root:
   x = [-2 ± √(4 - 4)] / 2
   x = [-2 ± √0] / 2
   x = [-2 ± 0] / 2

6) This gives two solutions:
   x = -2/2 = -1

Therefore, the solution is x = -1.

You can verify this by plugging -1 back into the original equation:
(-1)^2 + 2(-1) = 1 - 2 = -1 ✓
Answer with system prompt : Let's solve this step by step:

1. First, let's rearrange the equation to standard quadratic form
   • Move all terms to one side of the equation
   • Add 1 to both sides
   x^2 + 2x + 1 = 0

2. This looks like a perfect square trinomial. Can you recognize the patt

The first test, LLM gives the answer and shows how it solved step by step. <br>
But the second answer is adjusted based on LLM is playing "a patient math tutor" and guides the user via questions. <br>

<h2>How does temperature influences generated completion?</h2>


In [8]:
# Start a new conversation
client.clear_history()
user_prompt = "Generate a one sentence unique book idea."
client.add_user_message(user_prompt)

# Keep temperature at the lower value
i=0
temperature=0.1
while i < 3:
    answer = client.gen_completion(temperature=temperature)
    print(f"Answer[{i}] with T={temperature} : \n{answer}\n")
    i += 1

Answer[0] with T=0.1 : 
A reclusive botanist discovers a rare plant that can communicate telepathically, revealing ancient secrets about humanity's forgotten connection to the natural world.

Answer[1] with T=0.1 : 
A reclusive botanist discovers a rare plant that can communicate telepathically, revealing ancient secrets about humanity's forgotten connection to the natural world.

Answer[2] with T=0.1 : 
A reclusive botanist discovers a rare plant that can communicate telepathically, revealing ancient secrets about humanity's forgotten connection to the natural world.



Due to low temperature, all generations look similar.

In [9]:
# Increase Temperature tp 0.95 for diverse completions
i=0
temperature=1.0
while i < 3:
    answer = client.gen_completion(temperature=temperature)
    print(f"Answer[{i}] with T={temperature} : \n{answer}\n")
    i += 1

Answer[0] with T=1.0 : 
A disgraced quantum physicist discovers she can communicate with her alternate selves across parallel universes, each living a radically different life, and must collaborate with these versions of herself to prevent a catastrophic interdimensional collapse.

Answer[1] with T=1.0 : 
A former cryptographer discovers an ancient encryption method that allows her to communicate with parallel versions of herself across different timelines, uncovering a cosmic conspiracy that threatens the fabric of reality.

Answer[2] with T=1.0 : 
A skilled botanist discovers a rare plant that can communicate telepathically, revealing ancient secrets about humanity's forgotten connection to the natural world.



The first two ideas are different but the last book idea is the same as the first set of book ideas generated.

In [10]:
client.clear_history()
user_prompt = "Generate database query to show what percentage of daily active users were using a given feature. Ensure that output is in SQL."
client.add_user_message(user_prompt)
client.add_assistant_message("```SQL")

# stop sequences, to stop generation
stop_sequences=["```"]
text = client.gen_completion(stop_sequences=stop_sequences)
text

"\nWITH daily_active_users AS (\n    SELECT \n        DATE(login_timestamp) AS active_date,\n        COUNT(DISTINCT user_id) AS total_daily_active_users\n    FROM user_logins\n    GROUP BY DATE(login_timestamp)\n),\n\nfeature_users AS (\n    SELECT \n        DATE(feature_usage_timestamp) AS feature_date,\n        COUNT(DISTINCT user_id) AS feature_active_users\n    FROM feature_usage_log\n    WHERE feature_name = 'target_feature'\n    GROUP BY DATE(feature_usage_timestamp)\n)\n\nSELECT \n    f.feature_date,\n    f.feature_active_users,\n    d.total_daily_active_users,\n    ROUND(\n        (f.feature_active_users * 100.0) / d.total_daily_active_users, \n        2\n    ) AS feature_usage_percentage\nFROM feature_users f\nJOIN daily_active_users d ON f.feature_date = d.active_date\nORDER BY f.feature_date;\n"

In [11]:
from IPython.display import Markdown
Markdown(text)


WITH daily_active_users AS (
    SELECT 
        DATE(login_timestamp) AS active_date,
        COUNT(DISTINCT user_id) AS total_daily_active_users
    FROM user_logins
    GROUP BY DATE(login_timestamp)
),

feature_users AS (
    SELECT 
        DATE(feature_usage_timestamp) AS feature_date,
        COUNT(DISTINCT user_id) AS feature_active_users
    FROM feature_usage_log
    WHERE feature_name = 'target_feature'
    GROUP BY DATE(feature_usage_timestamp)
)

SELECT 
    f.feature_date,
    f.feature_active_users,
    d.total_daily_active_users,
    ROUND(
        (f.feature_active_users * 100.0) / d.total_daily_active_users, 
        2
    ) AS feature_usage_percentage
FROM feature_users f
JOIN daily_active_users d ON f.feature_date = d.active_date
ORDER BY f.feature_date;


DAU: Calculates Daily Active Users from user_logins table by counting the unique number of user logins in a given date<br>
DFU: Calculates Daily Feature Usage from feature_user_log table by counting the unique number of user identifiers in a given date<br>
feature_usage_percentage: calculated by using daily numbers in DAU and DFU <br>

## References
1. [Handling Stop Reasons](https://docs.claude.com/en/api/handling-stop-reasons)